In [ ]:
import numpy as np
from unyt import Msun, Myr, Angstrom
from astropy.cosmology import LambdaCDM
from uncertainties import unumpy
import pandas as pd

from synthesizer.grid import Grid
from synthesizer.particle import Galaxy
from synthesizer.instruments.filters import FilterCollection

from synthesizer.emission_models import PacmanEmission
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.emissions.utils import (
    Ha,
    Hb, 
    O2b,
    O2r,
    O3r,
    O3b,
    O3,
)

np.random.seed(680010)

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from utilities import (
    compute_metallicity_dust_correction,
    compute_metallicity_nodust_correction
)

from obs_data_plots import (
    plot_met_data,
    plot_rebels_data
)

cosmo = LambdaCDM(Om0=0.307, Ode0=0.693, H0=67.77, Ob0=0.04825)
plt.style.use('styles/style.mplstyle')

In [ ]:
def plot_line_ratios(line_int, line_emergent, grid_type, nsigma, req_lines):
    
    key, value = grid_type
    
    fig, axs = plt.subplots(nrows = 2, ncols = 1, figsize=(5, 7), sharex=False, sharey=False, facecolor='w', edgecolor='k')
    axs = axs.ravel()
    twinxaxs = [ax.twinx() for ax in axs]
    twinyaxs = [ax.twiny() for ax in axs]

    line_gals_int, line_gals_att = {}, {}

    markers = ['s', 'H', '*', 'o']
    custom = [Line2D([], [], marker=ii, markersize=8, color='dodgerblue', linestyle='None') for ii in markers]

    for jj in range(4):
        
        tmp = {value: line_int[jj][key].luminosity.sum() for key, value in req_lines.items()}
        tmp['R23'] = (tmp['[OIII]4959']+tmp['[OIII]5007']+tmp['[OII]3727,29'])/tmp['Hbeta']
        tmp['R3'] = tmp['[OIII]5007']/tmp["Hbeta"]
        tmp['Ne3O2'] = tmp['NeIII3869']/tmp['[OII]3727,29']
        tmp['O32'] = tmp['[OIII]5007']/tmp['[OII]3727,29']
        
        Ha_int      = line_int[jj]['H 1 6562.80A'].luminosity.sum()
        Hb_int      = line_int[jj]['H 1 4861.32A'].luminosity.sum()
        O2b_int     = line_int[jj]['O 2 3726.03A'].luminosity.sum()
        O2r_int     = line_int[jj]['O 2 3728.81A'].luminosity.sum()
        O2_int      = O2b_int + O2r_int
        O3b_int     = line_int[jj]['O 3 4958.91A'].luminosity.sum()
        O3r_int     = line_int[jj]['O 3 5006.84A'].luminosity.sum()
        O3_int      = line_int[jj]['O 3 4958.91A,O 3 5006.84A'].luminosity.sum()
        
        if jj==0:
            axs[0].scatter(np.log10(tmp['O32']), np.log10(tmp['R23']), color='black', label='Intrinsic', marker=markers[jj], s=40, zorder=10)
            axs[1].scatter(np.log10(tmp['Ne3O2']), np.log10(tmp['R3']), color='black', label='Intrinsic', marker=markers[jj], s=40, zorder=10) 
            
        else:
            axs[0].scatter(np.log10(tmp['O32']), np.log10(tmp['R23']), color='black', marker=markers[jj], s=40, zorder=10)    
            axs[1].scatter(np.log10(tmp['Ne3O2']), np.log10(tmp['R3']), color='black', marker=markers[jj], s=40, zorder=10)    

        if jj==0:
            line_gals_int = {jj: tmp}    
        else:
            line_gals_int[jj] = tmp   

        for ii in range(4):
            color = plt.cm.tab10(ii)   

            Ha      = line_emergent[jj][ii]['H 1 6562.80A'].luminosity.sum()
            Hb      = line_emergent[jj][ii]['H 1 4861.32A'].luminosity.sum()
            O2b     = line_emergent[jj][ii]['O 2 3726.03A'].luminosity.sum()
            O2r     = line_emergent[jj][ii]['O 2 3728.81A'].luminosity.sum()
            O2      = O2b + O2r
            O3b     = line_emergent[jj][ii]['O 3 4958.91A'].luminosity.sum()
            O3r     = line_emergent[jj][ii]['O 3 5006.84A'].luminosity.sum()
            O3      = line_emergent[jj][ii]['O 3 4958.91A,O 3 5006.84A'].luminosity.sum()  
            
            tmp = {value: line_emergent[jj][ii][key].luminosity.sum() for key, value in req_lines.items()}
            tmp['R23'] = (tmp['[OIII]4959']+tmp['[OIII]5007']+tmp['[OII]3727,29'])/tmp['Hbeta']
            tmp['R3'] = tmp['[OIII]5007']/tmp["Hbeta"]
            tmp['Ne3O2'] = tmp['NeIII3869']/tmp['[OII]3727,29']
            tmp['O32'] = tmp['[OIII]5007']/tmp['[OII]3727,29']
            tmp['R2'] = tmp['[OII]3727,29']/tmp["Hbeta"]
            if ii==0:
                line_gals_att[jj] = {ii: tmp}    
            else:
                line_gals_att[jj][ii] = tmp        
            
            if jj==0:
                axs[0].scatter(np.log10(tmp['O32']), np.log10(tmp['R23']), color=color, label=rF'$\sigma={nsigma[ii]}$', s=60, marker=markers[jj], alpha=0.8, zorder=1)
                axs[1].scatter(np.log10(tmp['Ne3O2']), np.log10(tmp['R3']), color=color, label=rF'$\sigma={nsigma[ii]}$', s=60, marker=markers[jj], alpha=0.8, zorder=1)
            else:
                axs[0].scatter(np.log10(tmp['O32']), np.log10(tmp['R23']), color=color, s=60, marker=markers[jj], alpha=0.8, zorder=1)
                axs[1].scatter(np.log10(tmp['Ne3O2']), np.log10(tmp['R3']), color=color, s=60, marker=markers[jj], alpha=0.8, zorder=1) 


    obs_OIIIr = unumpy.uarray(plot_rebels_data('[OIII]5007_flux'), plot_rebels_data('[OIII]5007_flux_error'))
    obs_OIIIb = unumpy.uarray(plot_rebels_data('[OIII]4959_flux'), plot_rebels_data('[OIII]4959_flux_error'))
    obs_OII = unumpy.uarray(plot_rebels_data('[OII]3727,29_flux'), plot_rebels_data('[OII]3727,29_flux_error'))
    obs_NeIII = unumpy.uarray(plot_rebels_data('[NeIII]3869+HeI+Hz_flux'), plot_rebels_data('[NeIII]3869+HeI+Hz_flux_error'))
    obs_Hb = unumpy.uarray(plot_rebels_data('H-beta_flux'), plot_rebels_data('H-beta_flux_error'))

    obs_x0 = unumpy.log10(obs_OIIIr/obs_OII)
    obs_y0 = unumpy.log10((obs_OIIIr + obs_OIIIb + obs_OII)/obs_Hb)

    obs_x1 = unumpy.log10(obs_NeIII/obs_OII)
    obs_y1 = unumpy.log10(obs_OIIIr/obs_Hb)

    twinxaxs[0].errorbar(unumpy.nominal_values(obs_x0), unumpy.nominal_values(obs_y0), xerr=unumpy.std_devs(obs_x0), yerr=unumpy.std_devs(obs_y0), label='REBELS (Rowland+2025)', color='grey', marker='d', ls='None', alpha=0.1, markersize=8)
    twinxaxs[1].errorbar(unumpy.nominal_values(obs_x1), unumpy.nominal_values(obs_y1), xerr=unumpy.std_devs(obs_x1), yerr=unumpy.std_devs(obs_y1), label='REBELS (Rowland+2025)', color='grey', marker='d', ls='None', alpha=0.1, markersize=8)

    df = pd.read_csv('./data/JWSTPRIMAL_All_red.csv', delimiter=';')
    df = df.dropna()
    
    req_columnns = ['OIII_5008', 'OIII_4960', 'OII_3727', 'Hbeta', 'NeIII_3870']
    df = df[(df[req_columnns] >= 0).all(axis=1)]

    obs_OIIIr = unumpy.uarray(df['OIII_5008'], df['err_OIII_5008'])
    obs_OIIIb = unumpy.uarray(df['OIII_4960'], df['err_OIII_4960'])
    obs_OII = unumpy.uarray(df['OII_3727'], df['err_OII_3727'])
    obs_Hb = unumpy.uarray(df['Hbeta'], df['err_Hbeta'])
    obs_NeIII = unumpy.uarray(df['NeIII_3870'], df['err_NeIII_3870'])
    
    obs_x = unumpy.log10(obs_OIIIr/obs_OII)
    obs_y = unumpy.log10((obs_OIIIr + obs_OIIIb + obs_OII)/obs_Hb)


    obs_x0 = unumpy.log10(obs_OIIIr/obs_OII)
    obs_y0 = unumpy.log10((obs_OIIIr + obs_OIIIb + obs_OII)/obs_Hb)

    obs_x1 = unumpy.log10(obs_NeIII/obs_OII)
    obs_y1 = unumpy.log10(obs_OIIIr/obs_Hb)

    ok = (unumpy.std_devs(obs_x0)<2) * (unumpy.std_devs(obs_y0)<2) * (unumpy.std_devs(obs_x1)<2) * (unumpy.std_devs(obs_y1)<2) 
    obs_x0 = obs_x0[ok]
    obs_y0 = obs_y0[ok]
    obs_x1 = obs_x1[ok]
    obs_y1 = obs_y1[ok]
    # ok = np.random.randint(0, len(obs_x), 60)
    # obs_x = obs_x[ok]
    # obs_y = obs_y[ok]

    twinxaxs[0].errorbar(unumpy.nominal_values(obs_x0), unumpy.nominal_values(obs_y0), xerr=unumpy.std_devs(obs_x0), yerr=unumpy.std_devs(obs_y0), label='PRIMAL (Heintz+2025)', color='grey', marker='^', ls='None', alpha=0.1, markersize=8)
    twinxaxs[1].errorbar(unumpy.nominal_values(obs_x1), unumpy.nominal_values(obs_y1), xerr=unumpy.std_devs(obs_x1), yerr=unumpy.std_devs(obs_y1), label='PRIMAL (Heintz+2025)', color='grey', marker='^', ls='None', alpha=0.1, markersize=8)

    axs[0].set_title(r''+value, fontsize=12)
    
    axs[0].set_xlabel(r'O$_{32}$', fontsize=15)
    axs[0].set_ylabel(r'R$_{23}$', fontsize=15)

    axs[1].set_xlabel(r'Ne3O2', fontsize=15)
    axs[1].set_ylabel(r'R$_{3}$', fontsize=15)

    xlim = [[-0.3,2], [-1,1]]
    ylim = [[0,1.5], [0,1.5]]
    
    axs[0].legend(fontsize=11, ncols=2, loc='upper right')    

    for ii in range(2):
         
        axs[ii].set_xlim(xlim[ii])
        axs[ii].set_ylim(ylim[ii])
        axs[ii].grid(ls='dotted')
        twinxaxs[ii].set_xlim(xlim[ii])
        twinxaxs[ii].set_ylim(ylim[ii])
        twinxaxs[ii].set_yticks([])
        twinxaxs[ii].set_yticklabels([])
        twinyaxs[ii].set_xticks([])
        twinyaxs[ii].set_xticklabels([])


    twinxaxs[0].legend(fontsize=11, loc='lower left', framealpha=0.3) 

    twinyaxs[1].legend(custom, [r'Varying only A$_{\mathrm{V}}$', 'Varying A$_{\mathrm{V}}$ and ages', 'Varying A$_{\mathrm{V}}$ and Z', 'Varying all'], framealpha=0.3, fontsize=12, loc="upper right", )

    fig.subplots_adjust(hspace=0.18)


    # twinyax.legend(custom, [r'Varying only A$_{\mathrm{V}}$', 'Varying A$_{\mathrm{V}}$ and ages', 'Varying A$_{\mathrm{V}}$ and Z', 'Varying all'], frameon=True, fontsize=12, loc="upper left")

    plt.savefig(F'app_plots/app_toy_R23_O32_R3_Ne3O2_{key}.png', dpi=300, bbox_inches='tight')   
    plt.close()
    
    return line_gals_int, line_gals_att

In [ ]:
def plot_met_Av(line_gals_int, line_gals_att, Av, logOH, nsigma, grid_type):
    
    key, value = grid_type
    
    fig, axs = plt.subplots(nrows = 4, ncols = 2, figsize=(6, 7), sharex=True, sharey=True, facecolor='w', edgecolor='k')
    axs = axs.ravel()
    twinaxs = [ax.twinx() for ax in axs]

    markers = ['s', 'H', '*', 'o']
    custom = [Line2D([], [], marker=ii, markersize=8, color='dodgerblue', linestyle='None') for ii in markers]

    for ii in range(4):
        axs[2*ii].axhline(y=logOH[ii], label='Mass-weighted', color='black', ls='dashed')
        axs[2*ii+1].axhline(y=logOH[ii], color='black', ls='dashed')


    # Varying only Av
    axs[0].scatter(0.0, compute_metallicity_dust_correction(line_gals_int[0], Balmerdust=False)[0], color='black', s=50, marker=markers[0])
    axs[1].scatter(0.0, compute_metallicity_nodust_correction(line_gals_int[0])[0], color='black', s=50, marker=markers[0], label=r'Intrinsic')

    # Varying Av and ages
    axs[2].scatter(0.0, compute_metallicity_dust_correction(line_gals_int[1], Balmerdust=False)[0], color='black', s=50, marker=markers[1])
    axs[3].scatter(0.0, compute_metallicity_nodust_correction(line_gals_int[1])[0], color='black', s=50, marker=markers[1])

    # Varying Av and Z
    axs[4].scatter(0.0, compute_metallicity_dust_correction(line_gals_int[2], Balmerdust=False)[0], color='black', s=50, marker=markers[2])
    axs[5].scatter(0.0, compute_metallicity_nodust_correction(line_gals_int[2])[0], color='black', s=50, marker=markers[2])


    # Varying all
    axs[6].scatter(0.0, compute_metallicity_dust_correction(line_gals_int[3], Balmerdust=False)[0], color='black', s=50, marker=markers[3])
    axs[7].scatter(0.0, compute_metallicity_nodust_correction(line_gals_int[3])[0], color='black', s=50, marker=markers[3])


    for jj in range(4):

        for ii in range(4):
            color = plt.cm.tab10(ii)

            axs[2*jj].scatter(Av[jj][ii], compute_metallicity_dust_correction(line_gals_att[jj][ii])[0], edgecolor=color, facecolor='None', s=80, marker=markers[jj])
            
            axs[2*jj].scatter(Av[jj][ii], compute_metallicity_dust_correction(line_gals_att[jj][ii], Balmerdust=False, Avdust=True, Av=Av[jj][ii])[0], color=color, s=40, marker=markers[jj], alpha=0.5)   

            if jj==1:
                axs[2*jj+1].scatter(Av[jj][ii], compute_metallicity_nodust_correction(line_gals_att[jj][ii])[0], edgecolor=color, s=80, marker=markers[jj], label=rF'$\sigma={nsigma[ii]}$')
            else:
                axs[2*jj+1].scatter(Av[jj][ii], compute_metallicity_nodust_correction(line_gals_att[jj][ii])[0], edgecolor=color, s=80, marker=markers[jj])



    for ax in axs:
        ax.grid(ls='dotted', alpha=0.5)
        ax.set_xticks(np.arange(0.1,1,0.2))
        ax.set_xlim(-0.03,0.7)
        # ax.set_ylim(7.1, 8.7)

    for ax in twinaxs:
        ax.set_xlim(-0.03,0.7)
        # ax.set_ylim(7.1, 8.7)
        ax.set_yticks([])
        ax.set_yticklabels([])

    axs[-1].set_xlabel(r'A$_{V}$', fontsize=12)
    axs[-2].set_xlabel(r'A$_{V}$', fontsize=12)
    # for ii in range(4):
    #     axs[2*ii].set_ylabel(r'$12+$log$_{10}$(O/H)', fontsize=12)
        

    axs[0].set_title(r'R23, O32 ('+value+')', fontsize=12)
    axs[1].set_title(r'R3, Ne3O2 ('+value+')', fontsize=12)

    twinaxs[0].legend([custom[0]], [r'Varying only A$_{\mathrm{V}}$'], loc='best', framealpha=0.2)
    axs[1].legend(frameon=True, loc='best', framealpha=0.2)

    twinaxs[2].legend([custom[1]], ['Varying A$_{\mathrm{V}}$ and ages'], loc='best', framealpha=0.2)
    axs[3].legend(frameon=True, loc='best', ncols=2, framealpha=0.2)

    twinaxs[4].legend([custom[2]], ['Varying A$_{\mathrm{V}}$ and Z'], loc='best', framealpha=0.2)

    twinaxs[6].legend([custom[3]], ['Varying all'], loc='best', framealpha=0.2)

    fig.text(0.05, 0.5, r'$12+$log$_{10}$(O/H)', va='center', fontsize=12, rotation='vertical')

    # fig.tight_layout()
    fig.subplots_adjust(hspace=0,wspace=0)

    plt.savefig(F'app_plots/app_toy_met_Av_{key}.png', dpi=300)
    plt.close()   

In [ ]:
# Define the grid
grid_names = [
    "bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps-fixed_U-m2p5",
    "bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps", 
    "bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps-fixed_U-m2",
    "bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps-fixed_U-m1p5", 
    "bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps-fixed_U-m1", 
    "bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps-fixed_nh-1", 
    "bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps-fixed_nh-2", 
    "bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps-fixed_nh-3", 
    "bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps-fixed_nh-3p5", 
    "bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps-fixed_nh-4",
    "bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps-fixed_nh-5",
    "bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps-fixed_nh-6",
    "bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps-fixed_U-m1p5_nh-3", 
    "bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps-fixed_U-m1p5_nh-4"
]
grid_dict = {
    "U-m2p5": "U $=10^{-2.5}$", 
    "default": "Default, U$_{\mathrm{ref}}=10^{-2}$", 
    "U-m2": "U $=10^{-2}$", 
    "U-m1p5": "U $=10^{-1.5}$", 
    "U-m1": "U $=10^{-1}$", 
    "nh-1": "n$_{\mathrm{H}}=10$ cm$^{-3}$", 
    "nh-2": "n$_{\mathrm{H}}=10^2$ cm$^{-3}$", 
    "nh-3": "n$_{\mathrm{H}}=10^3$ cm$^{-3}$", 
    "nh-3p5": "n$_{\mathrm{H}}=10^{3.5}$ cm$^{-3}$", 
    "nh-4": "n$_{\mathrm{H}}=10^4$ cm$^{-3}$",
    "nh-5": "n$_{\mathrm{H}}=10^5$ cm$^{-3}$",
    "nh-6": "n$_{\mathrm{H}}=10^6$ cm$^{-3}$",
    "U-m1p5_nh-3": "U $=10^{-1.5}$, n$_{\mathrm{H}}=10^3$ cm$^{-3}$",
    "U-m1p5_nh-4": "U $=10^{-1.5}$, n$_{\mathrm{H}}=10^4$ cm$^{-3}$"
}
grid_dir = "./data/"
grids = {}
for ii, key in enumerate(grid_dict.keys()):
    grids[key] = Grid(grid_names[ii], grid_dir=grid_dir)


In [ ]:
# Define required filter collection
Zsun = 0.014
req_lines = {'H 1 6562.80A': 'Halpha',
             'H 1 4861.32A': 'Hbeta',
             'O 2 3726.03A,O 2 3728.81A': '[OII]3727,29',
             'O 3 4958.91A': '[OIII]4959',
             'O 3 5006.84A': '[OIII]5007',
             'Ne 3 3868.76A': 'NeIII3869'
}
tophats = {
    "UV1500": {"lam_eff": 1500*Angstrom, "lam_fwhm": 300*Angstrom},
    "UV2800": {"lam_eff": 2800*Angstrom, "lam_fwhm": 300*Angstrom},
}

filters = FilterCollection(
    filter_codes=[f'Generic/Johnson.V'],
    tophat_dict=tophats,
    new_lam=grids["default"].lam
)

li = [
       Ha,
       Hb, 
       O2b,
       O2r,
       O3r,
       O3b,
       O3,
       'Ne 3 3868.76A'
   ]

In [ ]:
redshift = 6.
# Number of star particles
n = 100
ages = 10**np.random.uniform(0, np.log10(50), size=n) * Myr
Z = 10**np.random.uniform(-3.5, -2., size=n)
mass = 1e6*np.random.uniform(2, 5, n) * Msun 

# Dust distributions
max_tau = 7
mu, sigma = 0.1, 0.2
nsigma = np.arange(0,max_tau,2)
num_gals = len(nsigma)

# Vary everything
gal = Galaxy(redshift=redshift)
gal.load_stars(
    ages=ages,
    metallicities=Z,
    initial_masses=mass 
)
ok = gal.stars.ages<=10*Myr
Zmean = np.sum(gal.stars.initial_masses[ok] * gal.stars.metallicities[ok]) / np.sum(gal.stars.initial_masses[ok])
Agemean = np.sum(gal.stars.initial_masses[ok] * gal.stars.ages[ok]) / np.sum(gal.stars.initial_masses[ok])

print (Zmean, Agemean)

# Keep everything fixed
gal_const = Galaxy(redshift=redshift)
gal_const.load_stars(
    ages=np.ones(n) * Agemean,
    metallicities=np.ones(n) * Zmean,
    initial_masses=mass 
)

# Vary age
gal_vary_age = Galaxy(redshift=redshift)
gal_vary_age.load_stars(
    ages=ages,
    metallicities=np.ones(n) * Zmean,
    initial_masses=mass 
)

# Vary metallicity
gal_vary_Z = Galaxy(redshift=redshift)
gal_vary_Z.load_stars(
    ages=np.ones(n) * Agemean,
    metallicities=Z,
    initial_masses=mass 
)

all_gals = [gal_const, gal_vary_age, gal_vary_Z, gal]

In [ ]:
!rm app_plots/*

In [ ]:
tau_v = np.zeros((num_gals, n))
for ii in range(num_gals):
    tmp  = mu + np.random.normal(0, nsigma[ii], n)
    tmp[tmp<0] = 0.01
    tau_v[ii] = tmp

for key, value in grid_dict.items():
    
    grid = grids[key]
    
    # Property holders
    logOH = np.zeros(4)
    lum_int = np.zeros(4, dtype=object)
    line_int = np.zeros(4, dtype=object)

    line_emergent = np.zeros((4, num_gals), dtype=object)
    sed_emergent = np.zeros((4, num_gals), dtype=object)
    lum_att = np.zeros((4, num_gals))
    Av = np.zeros((4, num_gals))
    
    for jj, kk in enumerate(all_gals):

        print ("Galaxy number: ", jj)
        ok = kk.stars.ages<=10*Myr
        Zmean = np.sum(kk.stars.initial_masses[ok] * kk.stars.metallicities[ok]) / np.sum(kk.stars.initial_masses[ok])    
        logOH[jj] = 8.69 + np.log10(Zmean/Zsun)

        for ii in range(num_gals):
            emodel = PacmanEmission(
                grid=grid,
                tau_v=tau_v[ii],
                dust_curve=PowerLaw(),
                per_particle=True
            )    
        
            kk.stars.get_spectra(emission_model=emodel)    
            kk.stars.get_lines(li,
                emodel
            )        
            
            line_emergent[jj][ii] = kk.stars.lines['emergent']
            kk.stars.spectra['emergent'].get_fnu(cosmo=cosmo, z=redshift)   
            tmp = kk.stars.spectra['emergent'].get_photo_lnu(filters, verbose=False)
            lum_att[jj][ii] = tmp['Generic/Johnson.V']
            
            sed_emergent[jj][ii] = kk.stars.spectra['emergent']

        line_int[jj] = kk.stars.lines['intrinsic']    
        lum_int[jj] = kk.stars.spectra['intrinsic'].get_photo_lnu(filters, verbose=False)['Generic/Johnson.V']    

        Av[jj] = -2.5 * np.log10(lum_att[jj]/lum_int[jj])
        
    print (Av)    
    line_gals_int, line_gals_att = plot_line_ratios(line_int, line_emergent, [key, value], nsigma, req_lines)
    # plot_met_Av(line_gals_int, line_gals_att, Av, logOH, nsigma, [key, value])
        